In [ ]:
## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode


## Install NeMo
BRANCH = 'r1.4.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml
!pip install matplotlib==3.1.3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.4.0) to /tmp/pip-install-29_1y34a/nemo-toolkit_0f4bc849c10a4f14992eeeb3a7da5b81
  Running command git clone -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-29_1y34a/nemo-toolkit_0f4bc849c10a4f14992eeeb3a7da5b81
  Running command git checkout -b r1.4.0 --track origin/r1.4.0
  Switched to a new branch 'r1.4.0'
  Branch 'r1.4.0' set up to track remote branch 'r1.4.0' from 'origin'.
  Using cached matplotlib-3.5.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
  Attempting uninstall: matplotlib
    Found existing

In [ ]:
!pip install torchtext==0.10.1
!pip install matplotlib==3.1.3

  Using cached torchtext-0.10.1-cp37-cp37m-manylinux1_x86_64.whl (7.6 MB)
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [ ]:
# resolve different versions problem
!pip install torchtext==0.10.1
!pip install torchaudio==0.9.1

  Using cached torchaudio-0.9.1-cp37-cp37m-manylinux1_x86_64.whl (1.9 MB)
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111


In [ ]:
# Папка, где будет размещаться датасет
data_dir = '/content/'

In [ ]:
import glob
import os
import subprocess
import tarfile
import wget

# Загрузка датасета Golos
def load(golos_url, fname):
    if not os.path.exists(os.path.join(data_dir, fname)):
        golos_path = wget.download(golos_url, os.path.join(data_dir, fname))
        print(f"Dataset downloaded at: {golos_path}")
    else:
        print("Tarfile already exists.")
        golos_path = os.path.join(data_dir, fname)
    return golos_path

In [ ]:
# download dataset for training
train_farfield_path = load("https://sc.link/1Z3", "train_farfield.tar")

if not os.path.exists(os.path.join(data_dir, '/train_farfield/')):
    tar = tarfile.open(train_farfield_path)
    tar.extractall(path=data_dir + 'train_farfield/')
else:
    print("File already exists.")

Dataset downloaded at: /content/train_farfield.tar


In [ ]:
%rm -rf train_farfield.tar

In [ ]:
# download test files

test_path = load("https://sc.link/Kqr", "test.tar")

if not os.path.exists(os.path.join(data_dir, '/test/')):
    tar = tarfile.open(test_path)
    tar.extractall(path=data_dir)
else:
    print("File already exists.")

Dataset downloaded at: /content/test.tar


In [ ]:
%rm -rf test.tar

In [ ]:
# NeMo's "core" package
import nemo

# NeMo's ASR collection - this collections contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr

[NeMo W 2021-11-29 16:32:46 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2021-11-29 16:32:49 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali._AudioTextDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


## Исползование предобученной модели

In [ ]:
load("https://sc.link/ZMv", "QuartzNet15x5_golos.nemo")

asr_model = nemo_asr.models.EncDecCTCModel.restore_from(os.path.join(data_dir, "QuartzNet15x5_golos.nemo"))

Dataset downloaded at: /content/QuartzNet15x5_golos.nemo


[NeMo W 2021-11-29 16:33:12 modelPT:131] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: train/golos_and_mcv.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 64
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    num_workers: 20
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    parser: ru
    
[NeMo W 2021-11-29 16:33:12 modelPT:138] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a

[NeMo I 2021-11-29 16:33:12 features:262] PADDING: 16
[NeMo I 2021-11-29 16:33:12 features:279] STFT using torch
[NeMo I 2021-11-29 16:33:19 save_restore_connector:143] Model EncDecCTCModel was successfully restored from /content/QuartzNet15x5_golos.nemo.


### Transfer learning

In [ ]:
# Building manifest file for training

import json


# Function to build a manifest and get id:transcriptions from training files
def build_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0][:-8]
    farfield_true_trans = {}
    with open(manifest_rel, 'r') as rel:
        with open(manifest_abs, 'w') as abs:
            for info in rel:
                metadata = json.loads(info)
                if 'farfield' in metadata['audio_filepath']:
                    if metadata['id'] == '375e547e51a71f9e18b811dd89244baf':
                        continue
                    # нужно ли id+.wav ?
                    metadata["audio_filepath"] = os.path.join(manifest_path, metadata["audio_filepath"][:-4] + 'wav')
                    farfield_true_trans[metadata['id'] + '.wav'] = metadata['text']
                    json.dump(metadata, abs)
                    abs.write('\n')
    return farfield_true_trans


# Building Manifests
print("******")
train_manifest = data_dir + 'train_farfield/train/farfield/manifest.jsonl'
train_manifest_farfield = data_dir + 'train_farfield/train/farfield/manifest_farfield.jsonl'

if not os.path.isfile(train_manifest_farfield):
    farfield_true_trans = build_manifest(train_manifest, train_manifest_farfield)

print("train_manifest_farfield path: ", train_manifest_farfield)

******
train_manifest_farfield path:  /content/train_farfield/train/farfield/manifest_farfield.jsonl


In [ ]:
! head "/content/train_farfield/train/farfield/manifest_farfield.jsonl"

{"id": "855e01d6ba9a4aa59950d62037b87709", "audio_filepath": "/content/train_farfield/train/farfield/855e01d6ba9a4aa59950d62037b87709.wav", "text": "\u0430\u0444\u0438\u043d\u0430 \u0445\u043e\u0442\u0435\u043b\u0430 \u043d\u043e\u043c\u0435\u0440 \u0442\u0435\u043b\u0435\u0444\u043e\u043d\u0430 \u043f\u043e\u043c\u0435\u043d\u044f\u0442\u044c \u043a\u0430\u0440\u0442\u044b", "duration": 5.4596875}
{"id": "f6075fae0c592183f107cf781117d9b5", "audio_filepath": "/content/train_farfield/train/farfield/f6075fae0c592183f107cf781117d9b5.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u0445\u043e\u0447\u0443 \u043f\u043e\u0433\u043e\u0432\u043e\u0440\u0438\u0442\u044c \u0441 \u0434\u0436\u043e\u0439", "duration": 4.15125}
{"id": "2599a6cb55795b38ddd80b849d8670bc", "audio_filepath": "/content/train_farfield/train/farfield/2599a6cb55795b38ddd80b849d8670bc.wav", "text": "\u0434\u0436\u043e\u0439 \u043a\u0430\u043a\u0430\u044f \u0432\u0430\u043b\u044e\u0442\u0430 \u0432 \u043a\u0430\u043c\u0431\u04

In [ ]:
file_names = []
for file_name in farfield_true_trans.keys():
    file_names.append(data_dir + '/train_farfield/train/farfield/' + file_name)

transcriptions = asr_model.transcribe(paths2audio_files=file_names)

for fname, transcription in zip(file_names, transcriptions):
    print(f"Audio in {fname} was recognized as: {transcription}")

Transcribing:   0%|          | 0/31001 [00:00<?, ?it/s]

[NeMo W 2021-11-29 17:04:20 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-11-29 17:04:21 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
    To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
      return torch.floor_divide(self, other)
    


Выходные данные были обрезаны до нескольких последних строк (5000).
Audio in /content//train_farfield/train/farfield/9bb602f9a9712806e32ea8919e286717.wav was recognized as: пополни мой номер телефона на двести рублей пожалуйста
Audio in /content//train_farfield/train/farfield/4ea7ba84c3f02211bf5239b22863037c.wav was recognized as: сбер пополни сотовую связь
Audio in /content//train_farfield/train/farfield/2590a2e95aff2c2b58ff0e4cf77e8f59.wav was recognized as: салют слишком громкий звук давай его понизим
Audio in /content//train_farfield/train/farfield/ac3684e5f1eb79145ea7a16c67b59843.wav was recognized as: джой всадник без головы
Audio in /content//train_farfield/train/farfield/a7f23982ced21d4894087de3830f3a27.wav was recognized as: афина можешь перевести сто рублей сто рублей моему другу андрею
Audio in /content//train_farfield/train/farfield/08846a0d75e0f0b417bfe2ec320b3bf0.wav was recognized as: джой противоположность невежества
Audio in /content//train_farfield/train/farfield/2999

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Выходные данные были обрезаны до нескольких последних строк (5000).
Audio in /content//train_farfield/train/farfield/6309e0d059bdcc878e6be62a524ba85e.wav was recognized as: афина долгота мост
Audio in /content//train_farfield/train/farfield/a4ebda3914aadd73e85b289ec885b2ea.wav was recognized as: у меня прострочен платеж по кредиту я пополнила карту но деньги с нее
Audio in /content//train_farfield/train/farfield/993ad48dc6fba6a657c4d19f80cafe14.wav was recognized as: установи будильник на завтра на девять утра
Audio in /content//train_farfield/train/farfield/506fafe2102753d2f92c212152760a06.wav was recognized as: сбер показать канлы бокс тв плюс
Audio in /content//train_farfield/train/farfield/0534315bbacff89478379226b14821c7.wav was recognized as: как и погода замечательно
Audio in /content//train_farfield/train/farfield/d49167cc34b8e16593ebbc539720f065.wav was recognized as: сбер прошу ваш банк изменить условия договора
Audio in /content//train_farfield/train/farfield/e692e6f51ecdc7c

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Выходные данные были обрезаны до нескольких последних строк (5000).
Audio in /content//train_farfield/train/farfield/68e51ce1698e3bfbb9e3fbef49db694c.wav was recognized as: сбер а кто же еще
Audio in /content//train_farfield/train/farfield/8acd792250b3d69d7b9b6a5650739249.wav was recognized as: салют а если не секрет сколько тебе лет
Audio in /content//train_farfield/train/farfield/1c645ed8b18d308a5f0a992959ccc4a6.wav was recognized as: сбер средняя продолжительность жизни белис
Audio in /content//train_farfield/train/farfield/837b74bdf9b9be9b766d0a180cb2231c.wav was recognized as: салют запишите меня на прием в больницу
Audio in /content//train_farfield/train/farfield/e49ed2ee43c2d9c68355d35a887fe558.wav was recognized as: салют чем знаменит город ухань
Audio in /content//train_farfield/train/farfield/95d96fb8c11d8346101291de18e0bd7c.wav was recognized as: джой что будет с россией через десять лет
Audio in /content//train_farfield/train/farfield/44cdd34259b17fc5b918d93635ba16ea.wav wa

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
trans_by_model = {}
for file_name, transcription in zip(file_names, transcriptions):
    trans_by_model[file_name.split('/')[-1][:-4]] = transcription

In [ ]:
!pip install Levenshtein

     |████████████████████████████████| 110 kB 4.3 MB/s 


In [ ]:
import Levenshtein as levenshtein

In [ ]:
fales_trans_files = []
for file_name in farfield_true_trans.keys():
    file_id = file_name.split('.')[0]
    ratio_lev = levenshtein.ratio(farfield_true_trans[file_name], trans_by_model[file_id])
    if ratio_lev < 0.95:
        fales_trans_files.append(file_id)

In [ ]:
# del
print(len(fales_trans_files))

4223


In [ ]:
# Building manifest file for transfer learning on bad files


# Function to build a manifest
def build_false_manifest(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    print(manifest_path)
    with open(manifest_rel, 'r') as rel:
        with open(manifest_abs, 'w') as abs:
            for info in rel:
                metadata = json.loads(info)
                if metadata['id'] in fales_trans_files:
                    metadata["audio_filepath"] = os.path.join(manifest_path, metadata["audio_filepath"])
                    json.dump(metadata, abs)
                    abs.write('\n')


false_train_manifest = "/content/train_farfield/train/farfield/false_manifest.jsonl"
build_false_manifest(train_manifest_farfield, false_train_manifest)
print("false_manifest path: ", false_train_manifest)

/content/train_farfield/train/farfield
false_manifest path:  /content/train_farfield/train/farfield/false_manifest.jsonl


In [ ]:
! head "/content/train_farfield/train/farfield/false_manifest.jsonl"

{"id": "3f0d173552419362cf7d9e4b57ec26b8", "audio_filepath": "/content/train_farfield/train/farfield/3f0d173552419362cf7d9e4b57ec26b8.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u043d\u0430\u0439\u0434\u0438 \u0434\u043e\u0440\u0430\u043c\u0443", "duration": 3.4203125}
{"id": "3ab23d551a75661e834636ff40811eaa", "audio_filepath": "/content/train_farfield/train/farfield/3ab23d551a75661e834636ff40811eaa.wav", "text": "\u0441\u0430\u043b\u044e\u0442 \u043a\u0430\u0440\u0442\u0430 \u043e\u043a\u0435\u0430\u043d\u0438\u0438", "duration": 4.437375}
{"id": "5ffa87b9132dc49edb3c64c15a136416", "audio_filepath": "/content/train_farfield/train/farfield/5ffa87b9132dc49edb3c64c15a136416.wav", "text": "\u0434\u0436\u043e\u043d \u0431\u0430\u043d\u043a\u043e\u043c\u0430\u0442 \u043d\u0430\u0439\u0434\u0438", "duration": 4.6200625}
{"id": "64f8fe4f96a3c998f1f5f80d4f39c27a", "audio_filepath": "/content/train_farfield/train/farfield/64f8fe4f96a3c998f1f5f80d4f39c27a.wav", "text": "\u0434\u0436\u043e\u0

In [ ]:
# Building manifest file for test


# Function to build a manifest
def build_manifest_test(manifest_rel, manifest_abs):
    manifest_path = os.path.split(os.path.abspath(manifest_rel))[0]
    print(manifest_path)
    with open(manifest_rel, 'r') as rel:
        with open(manifest_abs, 'w') as abs:
            for info in rel:
                metadata = json.loads(info)
                metadata["audio_filepath"] = os.path.join(manifest_path, metadata["audio_filepath"])
                json.dump(metadata, abs)
                abs.write('\n')
                
# Building Manifests
print("******")
test_manifest = os.path.join(data_dir, 'test/farfield/manifest.jsonl')
test_manifest_farfield = os.path.join(data_dir, 'test/farfield/manifest_farfield.jsonl')

if not os.path.isfile(test_manifest_farfield):
    build_manifest_test(test_manifest, test_manifest_farfield)

print("test_manifest_farfield path: ", test_manifest_farfield)

******
test_manifest_farfield path:  /content/test/farfield/manifest_farfield.jsonl


In [ ]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=1, max_epochs=1, precision=16)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
params_model = asr_model._cfg

# --- Config Information ---#
# try:
#     from ruamel.yaml import YAML
# except ModuleNotFoundError:
#     from ruamel_yaml import YAML
# config_path = '/usr/local/lib/python3.7/dist-packages/nemo/collections/asr/models/configs/'

# yaml = YAML(typ='safe')
# with open(config_path) as f:
#     params = yaml.load(f)
# print(params)
params_model['train_ds']['parser']='base'
params_model['validation_ds']['parser']='base'
params_model['test_ds']['parser']='base'
params_model['train_ds']['manifest_filepath'] = false_train_manifest
params_model['train_ds']['batch_size']=4
params_model['validation_ds']['manifest_filepath'] = test_manifest_farfield
params_model['validation_ds']['batch_size']=4
params_model['test_ds']['manifest_filepath'] = test_manifest_farfield
params_model['test_ds']['batch_size']=4

In [ ]:
import copy
from omegaconf import DictConfig

new_opt = copy.deepcopy(params_model['optim'])
new_opt['lr'] = 0.0006
asr_model.setup_optimization(optim_config=DictConfig(new_opt))
# And then you can invoke trainer.fit(first_asr_model)

[NeMo W 2021-11-29 18:25:35 modelPT:436] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-29 18:25:35 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0006
        weight_decay: 0.001
    )


[NeMo W 2021-11-29 18:25:35 lr_scheduler:605] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.9, 0.98]
     eps: 1e-08
     grad_averaging: False
     lr: 0.0006
     weight_decay: 0.001
 ), None)

In [ ]:
asr_model.setup_training_data(train_data_config=params_model['train_ds'])
asr_model.setup_test_data(test_data_config=params_model['test_ds'])

# Point to the new validation data for fine-tuning
asr_model.setup_validation_data(val_data_config=params_model['validation_ds'])

[NeMo I 2021-11-29 18:26:25 collections:173] Dataset loaded with 1069 files totalling 1.01 hours
[NeMo I 2021-11-29 18:26:25 collections:174] 3 files were filtered totalling 0.01 hours


[NeMo W 2021-11-29 18:26:25 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


[NeMo I 2021-11-29 18:26:25 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 18:26:25 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2021-11-29 18:26:25 collections:173] Dataset loaded with 1916 files totalling 1.41 hours
[NeMo I 2021-11-29 18:26:25 collections:174] 0 files were filtered totalling 0.00 hours


In [ ]:
# Bigger batch-size = bigger throughput
# params['model']['validation_ds']['batch_size'] = 16
# Setup the test data loader and make sure the model is on GPU
# first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

WER = 0.8412073490813649


In [ ]:
trainer_add = pl.Trainer(gpus=1, max_epochs=6, precision=16)

Using native 16bit precision.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer_add.fit(asr_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2021-11-29 18:33:34 modelPT:436] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2021-11-29 18:33:34 modelPT:544] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0006
        weight_decay: 0.001
    )


[NeMo W 2021-11-29 18:33:34 lr_scheduler:605] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 34.9 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
18.9 M    Trainable params
0         Non-trainable params
18.9 M    Total params
75.718    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

[NeMo W 2021-11-29 18:33:34 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      cpuset_checked))
    


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
asr_model.cuda()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = asr_model._wer.compute()
        asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

WER = 0.7728465759961823
